## Manifold learning (continued) + Intro to Reinforcement Learning

#### 1.1. Manifold learning (continued)

In this first exercise, we will compare the power of typical classifiers before and after applying dimensionality reduction. Load the digit dataset from scikit learn by using the lines below.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets)

digits = datasets.load_digits(n_class=6)
X = digits.data
y = digits.target

Start by splitting the dataset into a training and a test part (lets say 10% test set) 


__1.1.1 No dimensionality reduction__ Apply Kmeans (either your own implementation or the scikit-learn implementation) to the original dataset.  Compute the validation error


In [ ]:
# put your code here




__1.1.2. Multidimensional Scaling__ Apply MDS to the dataset to get a mapping of the data to 2D. Then apply K means and compute the validation error.

In [ ]:
# put your code here




#### 1.2. Introduction to reinforcement learning with Open AI 'gym'

In this second exercise, you will get to apply what we have learned on reinforcement learning. To get some intuition on Reinforcement learning, we will turn to the gym library from openAI. 

As a first exercise, we will focus on the simple 'Taxi-v2' game which is a highly  simplified version of a self driving cab. The objective in this game is for the cab to pick up passenger at some location and drop them at the right destination. In this game, we basically want the rewards/penalties to work as follows

 - We want to give the agent a reward when it drops the passengers at the right location 
 - We will give it a strong penalty when it drops a passenger at the wrong location
 - Finally, to motivate the agent to reach the destinations as fast as possible, we will add a small for each time step after which the agent hasn't reached the desired destination
 
 
At any time step, our agent can take any of the following 6 actions: go south, go north, go east, go west, pick up a passenger or drop him/her off. 
 
The cab can pick up/drop off a passenger at any of the four locations marked 'R', 'G', 'Y' and 'B'. 
 
 
If we add one additional state corresponding to the passanger actually being in the car, this gives a state space of dimension $5\times 5 \times (4+1) \times5$. In other at each time step $t$, the agent is in one of the $500$ states, with passenger being somewhere and having to go somewhere else, and it has to decide among the $6$ possible actions.  


In [1]:
import gym

env = gym.make("Taxi-v2").env

env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In the 'Taxi-v2' game, the walls (which the taxi cannot cross) are represented by the vertical bars. The columns represent possible pathways. 

In Gym we access the environment through the 'env' variable. In particular, one can use 'env' to perform the following actions

 - env.reset (reset the environement to a random state)
 - env.step(action): update the state of the environment. This line returns the following:
     - The description of the environment
     - The reward associated with the last action 
     - A parameter that indicates if the cab has successfully completed the pick up and drop off (i.e such an action is called an episode)
 - 'env.render', displays one frame of the environment
 

To communicate with the agent, i.e set or get the state of the environment, we will use the 
functions 

   - env.encode 
   - env.s
    
The first call encodes the a 4-tuple describing a given environment state (taxi row, taxi column, passenger index, destination index) into a single integer corresponding to that particular state


Play a little with the 'env.encode',  'env.s' and 'env.render' to display different states

In [ ]:
# put your code here



To each state in the environment is also associated a reward table which contains, for every possible state, the probability to move to that state, the number encoding the state, the associated reward and a boolean variable which is true if the action has been taken and false otherwise. I.e the table has the form {action: [(probability, nextstate, reward, done)]}. You can display the table associated to any given state by using 'env.P[yourstateNumber]'

In [ ]:
# display the table



__1.2.1 No Reinforcement__ As a first exercise, we will try to solve the cab problem without reinforcement (i.e taking random actions until the cab dropped the passenger). To do this, write an infinite while loop which takes random actions until the variable 'done' from the reward table has been set to true. Keep track of a penalty variable which you increase by 1 each time the cab incurs a -10 reward. 

Store the variables 'State', 'Action' and 'Reward' as well as the current frame (use current_frame = env.render(mode='ansi')) throughout the iterations. (E.g store everything in a dictionnary 'frames')

(hint: use env.action_space.sample() to sample some action)

In [ ]:
# put your code here



Display the evolution of your cab using your dictionnary 'frames' and the snippet below

In [ ]:
from IPython.display import clear_output
from time import sleep

for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

        

__1.2.2 Part I: Training through Reinforcement__ We now turn to reinforcement learning. The first thing we will do is to learn a Q table which encodes how much we value each action the agent takes at any given state. In Q-learning (which is a particular choice of reinforcement learning), the value of each action is updated as 

$$Q(\text{state}, \text{action}) \leftarrow  (1-\alpha)Q(\text{state}, \text{action}) + \alpha(\text{reward} + \gamma \max_a Q(\text{next state}, \text{all actions}))$$

You see that the value of an action is updated from the previous value, the immediate reward associated to the action and 
the opportunities given by the action (i.e the maximum potential reward among future actions)

The larger $\gamma$, the more we weight long run behaviors. The smaller the value of $\gamma$, the more we weight short term rewards

In [ ]:
The Q-table is initialized to $0$ using the lines below. 

In [ ]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [ ]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

To explore the (state, reward) space, we will write two nested loops. An outer for loop, lets say over 100000 iterations which runs over episodes (that is we change iterations when done has been set to True). In each iteration we reset the initial state at random using 'state = env.reset()'

Then inside the for, we will write a while loop which runs until done is set to True. Inside this while loop, we will perform the following actions:
    
   - We will randomly choose between either 
        
         1.Exploring the space : in this case, we simply take a random action with env.action_space.sample()
         
         2.Taking advantage of our previous knowledge. In this case we use the Q table to define our next action: np.argmax(q_table[state])
          
   - Then we make a step and return the corresponding state
     
     'next_state, reward, done, info = env.step(action)'

   - Given the previous state, the next state, the reward, the table, update the table following the rule above.  
    


In [ ]:
# put your code here


once you have trained the agent (i.e the Q table). Display the value of the table for a few states between 0 and 500

__1.2.3. Evaluation__ OK. Now that we have the Q-table, we can simply evaluate the agent (lets say on 100 episodes) and see how the reward evolves compared to the random case. Write a for loop over about 100 iterations. In each outer iteration, start with a random state, set done = False, then write a while loop until done = False, where for each iteration, you return the best action from the Q-table for the current state, then update the environement using that state. 

Also define a variable cost that you increment by 1 each time you get a -10 reward. 

You can save the frames and display them as in the random case. 

In [ ]:
# put your code here


